In [ ]:
# ============================================================
# Imports & Config
# ============================================================
import numpy as np
import pandas as pd
import sys
from pathlib import Path

ROOT_DIR = Path.cwd().parent
sys.path.append(str(ROOT_DIR))

from src.data_pipeline import oleo_farelo, oleo_palma, oleo_diesel, oil_share
from src.visualization import plot_ratio_std_plotly
from src.utils import apply_theme, date_range_picker, rsi, section



# ============================================================
# 1) Escolhas do usuário (mude aqui)
# ============================================================
RATIO_OLEO_FARELO = "Óleo/Farelo"          # ["Óleo/Farelo", "Óleo/Palma", "Óleo/Diesel", "Oil Share CME"]
RATIO_OLEO_PALMA = "Óleo/Farelo"          # ["Óleo/Farelo", "Óleo/Palma", "Óleo/Diesel", "Oil Share CME"]
RATIO_OLEO_DIESEL = "Óleo/Farelo"          # ["Óleo/Farelo", "Óleo/Palma", "Óleo/Diesel", "Oil Share CME"]

START_DATE  = "2025-01-01"           # string ok
END_DATE    = "2025-12-17"           # string ok (hoje)
SUBPLOT_OPT = "RSI"          # ["Rolling STD", "RSI"]
RSI_LEN     = 14
MA_WINDOWS  = [90]                   # ex: [20, 50, 90, 200] ou []

ROLLING_WINDOW_STD_DEFAULT = 90      # janela usada no subplot de STD

# ============================================================
# 2) Mapa de ratios (df, coluna)
# ============================================================
RATIOS = {
    "Óleo/Farelo": (oleo_farelo, "oleo_farelo"),
    "Óleo/Palma":  (oleo_palma,  "oleo_palma"),
    "Óleo/Diesel": (oleo_diesel, "oleo_diesel"),
    "Oil Share CME": (oil_share, "oil_share"),
}

df_of, y_col = RATIOS[RATIO_OLEO_FARELO]
df_op, y_col = RATIOS[RATIO_OLEO_PALMA]
df_od, y_col = RATIOS[RATIO_OLEO_DIESEL]

# ============================================================
# 3) Checagens + datas
# ============================================================

df_of["date"] = pd.to_datetime(df_of["date"], errors="coerce")
df_op["date"] = pd.to_datetime(df_op["date"], errors="coerce")
df_od["date"] = pd.to_datetime(df_od["date"], errors="coerce")

start_date = pd.to_datetime(START_DATE).date()
end_date   = pd.to_datetime(END_DATE).date()

subplot_key = "std" if SUBPLOT_OPT == "Rolling STD" else "rsi"

# ============================================================
# 4) Filtra + view
# ============================================================
mask_of = (df_of["date"].dt.date >= start_date) & (df_of["date"].dt.date <= end_date)
view_of = df_of.loc[mask_of, ["date", y_col]].dropna().sort_values("date")

mask_op = (df_op["date"].dt.date >= start_date) & (df_op["date"].dt.date <= end_date)
view_op = df_op.loc[mask_of, ["date", y_col]].dropna().sort_values("date")

mask_od = (df_od["date"].dt.date >= start_date) & (df_od["date"].dt.date <= end_date)
view_od = df_od.loc[mask_od, ["date", y_col]].dropna().sort_values("date")

if view.empty:
    print("Sem dados no período selecionado.")
else:
    fig = plot_ratio_std_plotly(
        x=view["date"],
        y=view[y_col],
        title=f"Relação {RATIO_LABEL}",
        ylabel=f"Relação {RATIO_LABEL}",
        rolling_window=ROLLING_WINDOW_STD_DEFAULT,
        label_series=RATIO_LABEL,
        subplot=subplot_key,
        rsi_len=RSI_LEN,
        rsi_fn=rsi,
        ma_windows=MA_WINDOWS,
    )

    fig.update_layout(
        title=dict(pad=dict(b=12), x=0.0, xanchor="left", y=0.98, yanchor="top"),
        margin=dict(t=80),
    )

    fig.show()
